Какие распредления мы смотрели, когда работали над гипотезой

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from google.colab import drive
drive.mount('/content/drive') #подключаем гугл коллаб

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
values = pd.read_csv('/content/drive/My Drive/EkoNiva/value_ver3.csv', encoding='1251', sep=';', decimal=',')
works = pd.read_csv('/content/drive/My Drive/EkoNiva/works_ver3.csv', encoding='1251', sep=';', decimal=',')
fields = pd.read_csv('/content/drive/My Drive/EkoNiva/fields_ver3.csv', encoding='1251', sep=';', decimal=',')
fields.shape, works.shape, values.shape

In [ ]:
import datetime
works['start_date'] = works['start_date'].map(lambda x : datetime.datetime.strptime(x, "%d.%m.%Y") if type(x) == str else x)
works['end_date'] = works['end_date'].map(lambda x : datetime.datetime.strptime(x, "%d.%m.%Y") if type(x) == str else x)

def str2float(x):
    if type(x) != str:
        return x
    if '.' in x:
        a = x.split('.')
    else:
        a = x.split(',')

    return int(a[0]) + int(a[1]) / 100
works['area'] = works['area'].map(str2float)
works = works[works['mech'] != "Admin MS SQL"]

In [ ]:
works[['field', 'work_type', 'mech']].groupby(['field', 'work_type']).transform('count')

In [ ]:
works.head()

In [ ]:
works[works['prod_purpose'] == 'товарная']['yield_fact'].dropna().mean(), works[works['prod_purpose'] != 'товарная']['yield_fact'].dropna().mean()

In [ ]:
fields = pd.read_csv('/content/drive/My Drive/EkoNiva/fields_ver3.csv', encoding='1251', sep=';', decimal=',')
fields = fields.drop_duplicates()

In [ ]:
fields['id'] = fields['season'].map(str)+' '+fields['field'].map(str)
works['id'] = works['season'].map(str)+' '+works['field'].map(str)
merged_works = works.merge(fields, on='id', how='left')

In [ ]:
dropp = ["Пар",'Луг', "Залежь, лес", "Залежь, кустарник", "Не используется, чужая земля", "Пастбище на пашне", "Пастбище естественное", "Аренда", "Животноводческий комплекс", "Загон", "Залежь", "Не занято", "Не используется", "Строительство", "Пастбище на пашне", "Пастбище естественное", "Площадка под корма"]
merged_works['crop'] = merged_works['crop'].apply(lambda x:x if x not in dropp else np.nan)
merged_works = merged_works.dropna(subset='crop')

In [ ]:
dtt = merged_works[merged_works['prod_purpose'] != 'товарная'].drop_duplicates(subset=['id', 'crop'])['crop'].value_counts()
menn = dtt.mean()
iqr = dtt.quantile(0.25) * 1.5
dtt = dtt[dtt > menn - iqr]
dtt.index.to_list()

In [ ]:
hui = merged_works.drop_duplicates(subset=['id', 'season_x', 'work_type', 'mech']).groupby(['id', 'season_x', 'work_type'])['mech'].agg('count')
hui

In [ ]:
merged_works = merged_works.merge(hui, on=['id', 'season_x', 'work_type'], how='left')
sns.boxplot(merged_works, hue='')

In [ ]:
sns.histplot(works[works['prod_dry_perc'] < 150]['prod_dry_perc'], bins=90)
plt.xlabel("Содержание сухого вещества в почве, %")
plt.title('Распределение содержания сухого вещества')

In [ ]:
sns.boxplot(dtt)

In [ ]:
works['prod_purpose'].value_counts()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.histplot(works[works['prod_purpose'] == 'товарная']['yield_fact'], ax=ax[0])
sns.histplot(works[works['prod_purpose'] != 'товарная']['yield_fact'], ax=ax[1])

In [ ]:
work_counts = works['work_type'].value_counts()

# Преобразование в процентные соотношения
work_percentages = (work_counts / work_counts.sum()) * 100
colors = plt.cm.Blues(np.linspace(0.3, 0.9, 5))  # Цвета из палитры Paired
#colors = plt.cm.Set3.colors    # Альтернатива: палитра Set3
#colors = plt.cm.viridis.colors  # Альтернатива: палитра Viridis
#colors = plt.cm.coolwarm.colors  # Альтернатива: палитра Coolwarm
# Создание круговой диаграммы
plt.figure(figsize=(7, 7))  # Размер диаграммы
plt.pie(work_percentages, labels=work_percentages.index, colors=colors ,autopct='%1.1f%%', startangle=140)

# Настройка внешнего вида
plt.title('Распределение типов работы (%)', pad=5, fontweight='bold', fontsize=15)
plt.subplots_adjust(top=5)
plt.rcParams['font.size'] = 11
plt.show()

In [ ]:
yield_23 = works[works['season'] == 2023]['yield_fact']
yield_24 = works[works['season'] == 2024]['yield_fact']
meann_d = yield_23.mean()
iqr_d = 3 * (yield_23.quantile(0.75) - yield_23.quantile(0.25))
works_ = works_[(works['depth'] >= meann_d - iqr_d) & (works['depth'] <= meann_d + iqr_d)]
f, ax = plt.subplots(1, 2, figsize=(12, 6))
sns.boxplot(yield_23, ax=ax[0])
sns.boxplot(yield_24, ax=ax[1])

In [ ]:
dt = works[works['work_type'] == 'Уборка'].groupby(['field', 'start_date'])['mech'].count()
dt.head()

In [ ]:
dt.value_counts().head()

In [ ]:
sns.histplot(dt, bins=20)
plt.xticks(np.arange(0, 22, 2))
plt.yscale('log')

In [ ]:
newWorks = works[works['work_type'] == 'Уборка']
newWorks.shape

In [ ]:
counts_past = []

for i, (field, date) in enumerate(zip(newWorks['field'], newWorks['start_date'])):
    ws = newWorks[(newWorks['start_date'] < date) & (newWorks['field'] == field)]['mech'].nunique()
    ws = int(ws)

    counts_past.append(ws)

    if (i + 1) % 10_000 == 0:
        print(f"{i+1} done")

newWorks['counts_mech_past'] = counts_past

In [ ]:
# кол-во количества смененных работяг сборщиков
sns.histplot(newWorks['counts_mech_past'], bins=30)
plt.title('Распределение количества уникальных работников на полях')

In [ ]:
newWorks['counts_mech_past'].describe()

In [ ]:
goodWorks = newWorks[newWorks['counts_mech_past'] <= 3]
badWorks = newWorks[newWorks['counts_mech_past'] > 3]
goodWorks.shape, badWorks.shape

In [ ]:
# plan and культура

def coor_cat_num(df, numeric, categ): # функция для подсчета корреляции для категор/номинал

    inGroups = 0
    betweenGroups = 0

    # общее среднее
    df_mean = df[numeric].mean()

    for val in df[categ].unique():
        # для каждого уникального значения берем столбец
        data = df[df[categ] == val][numeric].dropna()
        if len(data) == 0:
            continue

        # добавляем это во внутригрупповые
        inGroups += ((data - data.mean())**2).sum()
        # Это в межгрупповые
        betweenGroups += len(data) * (data.mean() - df_mean) ** 2
    # выводим
    return (betweenGroups / (inGroups + betweenGroups)) ** 0.5

In [ ]:
coor_cat_num(goodWorks[['counts_mech_past', 'yield_fact']], 'yield_fact', 'counts_mech_past')

In [ ]:
badWorks[['counts_mech_past', 'yield_fact']].corr()

In [ ]:
goodWorks['yield_fact'].mean(), badWorks['yield_fact'].mean()

In [ ]:
def test_variance_ratio(sample1, sample2):
    """
    Выполняет тест отношения дисперсий (тест Фишера) для двух выборок.

    :param sample1: Первая выборка (список или массив)
    :param sample2: Вторая выборка (список или массив)
    :return: Статистика теста и p-значение
    """
    # Проверка, что выборки не пустые
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки не должны быть пустыми.")

    # Вычисляем дисперсии
    var1 = np.var(sample1, ddof=1)  # Используем ddof=1 для выборочной дисперсии
    var2 = np.var(sample2, ddof=1)

    # Вычисляем статистику F
    F = var1 / var2  # Применяем отношение дисперсий

    # Вычисляем p-значение
    dfn = len(sample1) - 1  # Степени свободы для первой выборки
    dfd = len(sample2) - 1  # Степени свободы для второй выборки
    p_value = stats.f.sf(F, dfn, dfd)  # Получаем p-значение

    return F, p_value

In [ ]:
def confidence_interval(data, confidence=0.95):
    # Вычисляем среднее и стандартное отклонение
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)
    n = len(data)

    # Находим критическое значение t
    t_critical = stats.t.ppf((1 + confidence) / 2, df=n-1)

    # Вычисляем стандартную ошибку
    standard_error = std_dev / np.sqrt(n)

    # Рассчитываем границы доверительного интервала
    margin_of_error = t_critical * standard_error
    lower_bound = mean - margin_of_error
    upper_bound = mean + margin_of_error

    return (lower_bound, upper_bound)

In [ ]:
def t_test_mannwhitney(group1, group2, n_samples=1000, sample_size=5000):
    """
    Сравнение двух групп с использованием T-теста или теста Манна-Уитни.

    Параметры:
    group1: array-like, первая группа данных
    group2: array-like, вторая группа данных
    n_samples: int, количество повторных семплов
    sample_size: int, размер выборки для каждой группы

    Возвращает:
    average_p_value: float, среднее p-value
    """

    p_values = []

    for _ in range(n_samples):
        # Семплирование
        sample1 = np.random.choice(group1, size=min(sample_size, len(group1)), replace=False)
        sample2 = np.random.choice(group2, size=min(sample_size, len(group2)), replace=False)

        # Проверка на нормальность и размер выборки
        if stats.shapiro(sample1).pvalue > 0.05 and stats.shapiro(sample2).pvalue > 0.05:
            # Используем T-тест
            t_stat, p_value = stats.ttest_ind(sample1, sample2)
        else:
            # Используем тест Манна-Уитни
            _, p_value = stats.mannwhitneyu(sample1, sample2, alternative='two-sided')

        p_values.append(p_value)

    # Возвращаем среднее p-value
    average_p_value = np.mean(p_values)
    return average_p_value

In [ ]:
def anova_kruskal(group1, group2, n_samples=1000, sample_size=5000):
    """
    Сравнение двух групп с использованием ANOVA или теста Краскела-Уоллиса.

    Параметры:
    group1: array-like, первая группа данных
    group2: array-like, вторая группа данных
    n_samples: int, количество повторных семплов
    sample_size: int, размер выборки для каждой группы

    Возвращает:
    average_p_value: float, среднее p-value
    """

    p_values = []

    for _ in range(n_samples):
        # Семплирование
        sample1 = np.random.choice(group1, size=min(sample_size, len(group1)), replace=False)
        sample2 = np.random.choice(group2, size=min(sample_size, len(group2)), replace=False)

        # Проверка на нормальность
        if stats.shapiro(sample1).pvalue > 0.05 and stats.shapiro(sample2).pvalue > 0.05:
            # Используем ANOVA
            f_stat, p_value = stats.f_oneway(sample1, sample2)
        else:
            # Используем тест Краскела-Уоллиса
            h_stat, p_value = stats.kruskal(sample1, sample2)

        p_values.append(p_value)

    # Возвращаем среднее p-value
    average_p_value = np.mean(p_values)
    return average_p_value


In [ ]:
def chi_square_test(data, column1, column2):
    """
    Выполняет тест хи-квадрат для проверки независимости двух категориальных переменных.

    Параметры:
    data: DataFrame, содержащий данные
    column1: str, имя первой категориальной переменной
    column2: str, имя второй категориальной переменной

    Возвращает:
    chi2_stat: float, значение статистики хи-квадрат
    p_value: float, p-значение
    """

    # Создание таблицы сопряженности
    contingency_table = pd.crosstab(data[column1], data[column2])

    # Выполнение теста хи-квадрат
    chi2_stat, p_value, _, _ = stats.chi2_contingency(contingency_table)

    return chi2_stat, p_value

In [ ]:
def proportion_test(success_a, total_a, success_b, total_b):
    # Объединенные пропорции
    p_a = success_a / total_a
    p_b = success_b / total_b
    p_combined = (success_a + success_b) / (total_a + total_b)

    # Стандартная ошибка
    se = np.sqrt(p_combined * (1 - p_combined) * (1/total_a + 1/total_b))

    # Z-статистика
    z = (p_a - p_b) / se
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))  # Двусторонний тест
    return z, p_value


success_a = 70  # Успехи в группе A
total_a = 100   # Всего в группе A
success_b = 60  # Успехи в группе B
total_b = 100   # Всего в группе B

z_stat, p_value = proportion_test(success_a, total_a, success_b, total_b)
print(f"Z-статистика для теста пропорций: {z_stat}, p-значение: {p_value}")


In [ ]:
t_test_mannwhitney(goodWorks['yield_fact'], badWorks['yield_fact'], 1000, 10000)